# Bootstrapping: Emittance uncertainty calculation

In [1]:
import statistics
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from random import seed

#### First, reading the raw data and displaying them in a dataframe.

In [2]:
I_meas, sigm = np.loadtxt('1_medida_emittance_planeX_E_147.8_MULTIBUNCH2.txt', skiprows=2, unpack=True)
current, err_current, sigma,err_sigma  = [],[],[],[]
for i in range(0, len(I_meas), 4):
    current.append(np.mean(I_meas[i:i+4]))
    err_current.append(np.std(I_meas[i:i+4]))
    sigma.append(np.mean(sigm[i:i+4]))
    err_sigma.append(np.std(sigm[i:i+4]))
Data = pd.DataFrame({' Current [A]':current,
                     '$\pm$ uncert. [A]': err_current,
                     'Beam size [mm]':np.array(sigma)*1000,
                     '$\pm$ uncert. [mm]': np.array(err_sigma)*1000})
Data

,Current [A],$\pm$ uncert. [A],Beam size [mm],$\pm$ uncert. [mm]
0,0.500139,0.000666,0.923498,0.008285
1,0.619840,0.000896,0.861830,0.007841
2,0.740328,0.000227,0.792794,0.007036
3,0.859553,0.000770,0.760187,0.008366
4,0.980407,0.000316,0.746459,0.000241
5,1.100016,0.000787,0.745868,0.000243
6,1.220637,0.000633,0.781876,0.004196
7,1.340480,0.000545,0.834189,0.008481
8,1.459988,0.000628,0.888755,0.007457
9,1.579458,0.000495,0.959203,0.001335


#### Setting the experimental set-up data

In [3]:
I2K1 = [-0.0089, 2.1891, 0.0493] # Fitting values to transform from current to quadrupole strength
DIST = 2.8775                    # Space (drift) between the quadrupole and the camera
QUAD_L = 0.112                   # Quadrupole length
light_speed = 299792458          # in m/s
electron_rest_en = 0.5109989461  # in MeV
energy = 147.8                   # beam energy in MeV

In [4]:
def _get_K1_from_I(energy, I_meas):
    '''Function to calculate the quadrupole strength using the quadrupole current'''
    kin_en = np.sqrt(energy*energy - electron_rest_en*electron_rest_en)
    return np.polyval(I2K1, I_meas)*light_speed/kin_en/1e6